In [1]:
from pulp import *

# Create a LP minimization problem
model = LpProblem("Project planning", LpMinimize)


/Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [2]:
# Define workers and their hourly rates
workers = {
    'ProjectManager': 72,  # Example hourly rate for the project manager
    'FrontendDeveloper': 60,
    'BackendDeveloper': 60,
    'DataScientist': 58,
    'DataEngineer': 72,
}

In [3]:
# Define tasks and their estimated times for different scenarios
activities = {
    'Describe product': {'ExpectedTime': 4},
    'Develop marketing strategy': {'ExpectedTime': 12},
    'Design brochure': {'ExpectedTime': 10},
    #'Develop product protoype': {'ExpectedTime': 60},
    'Requirements analysis': {'ExpectedTime': 8},
    'Software design': {'ExpectedTime': 12},
    'System design':{'ExpectedTime': 12}, 
    'Coding': {'ExpectedTime': 30},
    'Write documentation': {'ExpectedTime': 15},
    'Unit testing': {'ExpectedTime': 15},
    'System testing': {'ExpectedTime': 18},
    'Package deliverables': {'ExpectedTime': 6},
    'Survey potential market': {'ExpectedTime': 15},
    'Develop pricing plan': {'ExpectedTime': 9},
    'Develop implementation  plan': {'ExpectedTime': 12},
    'Write client proposal': {'ExpectedTime': 6}
}


In [4]:
# Create a list of the activities
activities_list = list(activities.keys())


In [5]:
# Create a dictionary of the activity precedences
precedences = {
            'Describe product': [],
            'Develop marketing strategy': [], 
            'Design brochure': ['Describe product'], 
            #'Develop product protoype': [],  
            'Requirements analysis': ['Describe product'], 
            'Software design': ['Requirements analysis'], 
            'System design': ['Requirements analysis'], 
            'Coding': ['Software design','System design'],
            'Write documentation': ['Coding'],
            'Unit testing': ['Coding'],
            'System testing' : ['Unit testing'],
            'Package deliverables': ['Write documentation','System testing'],
            'Survey potential market':['Develop marketing strategy','Design brochure'],
            'Develop pricing plan': ['Package deliverables','Survey potential market'],
            'Develop implementation  plan': ['Describe product','Package deliverables'],
            'Write client proposal': ['Develop pricing plan','Develop implementation  plan']
            }


In [6]:
# Define decision variables
x = pulp.LpVariable.dicts("Hours", [(worker, task) for worker in workers for task in activities],
                           lowBound=0)


In [7]:
#Objective function
model += pulp.lpSum(x[(worker, task)] * activities[task]['ExpectedTime'] for worker in workers for task in activities)

In [8]:
#Constraints
for task in activities:
    model += pulp.lpSum(x[(worker, task)] for worker in workers) >= activities[task]['ExpectedTime']

# Ensure that workers are available
# Example: Assuming 160 hours of work per worker
#for worker in workers:
    #model += pulp.lpSum(x[(worker, task)] for task in activities) <= 160


In [9]:
# Solve the linear programming problem
model.solve()

# Print the results
for worker in workers:
    for task in activities:
        if x[(worker, task)].varValue > 0:
            print(f"{worker} works {x[(worker, task)].varValue} hours on {task}")

print(f"Total Expected time: {pulp.value(model.objective)}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/05dedc5e478e4ca488ad1a1c05a72448-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/05dedc5e478e4ca488ad1a1c05a72448-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 20 COLUMNS
At line 171 RHS
At line 187 BOUNDS
At line 188 ENDATA
Problem MODEL has 15 rows, 75 columns and 75 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-15) rows, 0 (-75) columns and 0 (-75) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 2808
After Postsolve, objective 2808, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 2808 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions chang

In [40]:
# Create the LP problem
prob = LpProblem("Critical Path", LpMinimize)


In [41]:
# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}


In [42]:
# Add the constraints
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"


In [43]:
# Set the objective function
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if value(start_times[activity]) == 0:
        print(f"{activity} starts at time 0")
    if value(end_times[activity]) == max([value(end_times[activity]) for activity in activities_list]):
        print(f"{activity} ends at {value(end_times[activity])} days in duration")

# Print solution
print("\nSolution variable values:")
for var in prob.variables():
    if var.name != "_dummy":
        print(var.name, "=", var.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/3b7413bff42947a6aca1de4ad2d96d72-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/3b7413bff42947a6aca1de4ad2d96d72-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 40 COLUMNS
At line 127 RHS
At line 163 BOUNDS
At line 164 ENDATA
Problem MODEL has 35 rows, 32 columns and 70 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-35) rows, 0 (-32) columns and 0 (-70) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 869
After Postsolve, objective 869, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 869 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed 